In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('Pokemon.csv')

In [3]:
data

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


Columns description (it's crucial!)


- #: ID for each pokemon
- Name: Name of each pokemon
- Type 1: Each pokemon has a type, this determines weakness/resistance to attacks
- Type 2: Some pokemon are dual type and have 2
- Total: sum of all stats that come after this, a general guide to how strong a pokemon is
- HP: hit points, or health, defines how much damage a pokemon can withstand before fainting
- Attack: the base modifier for normal attacks (eg. Scratch, Punch)
- Defense: the base damage resistance against normal attacks
- SP Atk: special attack, the base modifier for special attacks (e.g. fire blast, bubble beam)
- SP Def: the base damage resistance against special attacks
- Speed: determines which pokemon attacks first each round

In [4]:
# fillna and drop useless cols

display(data.isnull().sum())
data['Type 2'] = data['Type 2'].fillna('No 2nd type')

data.drop(columns = ['#', 'Name'], inplace = True)

#               0
Name            0
Type 1          0
Type 2        386
Total           0
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

In [5]:
X = data.drop(columns = 'Legendary')
y = data['Legendary'].astype('int')

In [6]:
X.head()

,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
0,Grass,Poison,318,45,49,49,65,65,45,1
1,Grass,Poison,405,60,62,63,80,80,60,1
2,Grass,Poison,525,80,82,83,100,100,80,1
3,Grass,Poison,625,80,100,123,122,120,80,1
4,Fire,No 2nd type,309,39,52,43,60,50,65,1


In [7]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Legendary, dtype: int64

In [8]:
y.value_counts(normalize = True)

Legendary
0    0.91875
1    0.08125
Name: proportion, dtype: float64

# Make some default pipeline

In [9]:
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from category_encoders.leave_one_out import LeaveOneOutEncoder
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
import sklearn

In [10]:
# define cat_cols

cat_cols = ['Type 1', 'Type 2']

default_pipeline = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols = cat_cols)),
    ('scaler_', StandardScaler()),
    ('model_', SVC(kernel = 'linear'))]
)

In [11]:
cv_res1 = cross_validate(default_pipeline,
                        X,
                        y,
                        cv = 5,
                        scoring = 'f1',
                        n_jobs = -1,
                        return_train_score = True,
                        error_score = 'raise'
                       )

In [12]:
cv_res1

{'fit_time': array([0.01105404, 0.01224494, 0.01126075, 0.0116632 , 0.01072598]),
 'score_time': array([0.00291514, 0.00309396, 0.00364089, 0.00364494, 0.00297022]),
 'test_score': array([0.5       , 0.72727273, 0.47619048, 0.38095238, 0.64864865]),
 'train_score': array([0.71287129, 0.56097561, 0.6744186 , 0.7311828 , 0.72727273])}

In [13]:
cv_res1['train_score'].mean()

0.681344204901525

In [14]:
cv_res1['test_score'].mean()

0.5466128466128466

# Make pipeline more complicated

In [15]:
import pandas as pd
from category_encoders import LeaveOneOutEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.svm import SVC

In [16]:
# difficult pipeline

pipe_dif = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols = cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree = 4)),
    ('scaler_', StandardScaler()),
    ('model_', SVC(kernel = 'linear'))]
)

In [17]:
cv_res2 = cross_validate(pipe_dif,
                        X,
                        y,
                        cv = 5,
                        scoring = 'f1',
                        n_jobs = -1,
                        return_train_score = True
                       )

cv_res2

{'fit_time': array([0.05995226, 0.08819914, 0.0753231 , 0.07265306, 0.06223893]),
 'score_time': array([0.00960588, 0.00870991, 0.00950003, 0.00943995, 0.00846004]),
 'test_score': array([0.375     , 0.88888889, 0.5       , 0.66666667, 0.53658537]),
 'train_score': array([0.95145631, 0.89583333, 0.97142857, 0.96153846, 0.98076923])}

In [18]:
cv_res2['train_score'].mean()

0.9522051815498418

In [19]:
cv_res2['test_score'].mean()

0.5934281842818427

train_score - просто класс ! модель получилась сложная, только очевидно переобученная ...

согласны, узнали ?


# Introduce feature selectors

In [20]:
pipe_dif

Pipeline(steps=[('cat_encoder_', LeaveOneOutEncoder(cols=['Type 1', 'Type 2'])),
                ('poly_featurizer_', PolynomialFeatures(degree=4)),
                ('scaler_', StandardScaler()),
                ('model_', SVC(kernel='linear'))])

In [21]:
data_tr = pipe_dif[:-1]

In [22]:
data_tr

Pipeline(steps=[('cat_encoder_', LeaveOneOutEncoder(cols=['Type 1', 'Type 2'])),
                ('poly_featurizer_', PolynomialFeatures(degree=4)),
                ('scaler_', StandardScaler())])

In [23]:
help(pipe_dif)

Help on Pipeline in module sklearn.pipeline object:

class Pipeline(sklearn.utils.metaestimators._BaseComposition)
 |  Pipeline(steps, *, memory=None, verbose=False)
 |  
 |  A sequence of data transformers with an optional final predictor.
 |  
 |  `Pipeline` allows you to sequentially apply a list of transformers to
 |  preprocess the data and, if desired, conclude the sequence with a final
 |  :term:`predictor` for predictive modeling.
 |  
 |  Intermediate steps of the pipeline must be 'transforms', that is, they
 |  must implement `fit` and `transform` methods.
 |  The final :term:`estimator` only needs to implement `fit`.
 |  The transformers in the pipeline can be cached using ``memory`` argument.
 |  
 |  The purpose of the pipeline is to assemble several steps that can be
 |  cross-validated together while setting different parameters. For this, it
 |  enables setting parameters of the various steps using their names and the
 |  parameter name separated by a `'__'`, as in the 

In [24]:
X_tr = data_tr.fit_transform(X, y)

In [25]:
X_tr

array([[ 0.        , -0.40675128, -1.46809988, ..., -0.7760009 ,
        -0.82565808, -0.78950478],
       [ 0.        , -0.40675128, -1.46809988, ..., -0.75915834,
        -0.82278699, -0.78950478],
       [ 0.        , -0.40675128, -1.46809988, ..., -0.72921601,
        -0.81895887, -0.78950478],
       ...,
       [ 0.        ,  1.62491492, -1.46809988, ...,  1.0887111 ,
         2.05978425,  2.42568009],
       [ 0.        ,  1.62491492, -1.46809988, ...,  1.6661703 ,
         2.47322076,  2.42568009],
       [ 0.        , -0.03035286, -1.46809988, ...,  1.0887111 ,
         2.05978425,  2.42568009]])

In [26]:
print(f'data shape after transformation is {X_tr.shape}')

data shape after transformation is (800, 1001)


1k признаков - многовато, добавим в пайплайн селектор

## Фильтрационные методы

Суть таких методов в том, чтобы для каждого признака посчитать некоторую метрику "связи" с целевым признаком. И в результате оставить топ-K признаков согласно выбранной метрике.

В том числе:

 - статистика хи-квадрат
 - mutual_info_classif

In [27]:
from sklearn.feature_selection import f_classif, chi2, mutual_info_classif

In [28]:
k_best = 50

pipe = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols = cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree = 4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectKBest(score_func = mutual_info_classif, k = 50)), 
    ('model_', SVC(kernel = 'linear'))]
)

In [29]:
cv_res = cross_validate(pipe, X, y, cv = 5, scoring = 'f1', return_train_score = True, error_score = 'raise')
cv_res

{'fit_time': array([1.35355496, 1.28781199, 1.37500191, 1.24642611, 1.26759577]),
 'score_time': array([0.00921893, 0.00410223, 0.00515985, 0.00521803, 0.02109838]),
 'test_score': array([0.14285714, 0.64      , 0.71428571, 0.42105263, 0.6       ]),
 'train_score': array([0.79207921, 0.70103093, 0.74      , 0.75247525, 0.8       ])}

In [30]:
cv_res['train_score'].mean()

0.7571170766561192

In [31]:
# k best нужно подбирать

cv_res['test_score'].mean()

0.503639097744361

## Жадный метод отбора

In [32]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [33]:
rfe = RFE(LogisticRegression(max_iter = 1000), n_features_to_select = k_best, step = 30)

In [34]:
X_tr.shape

(800, 1001)

In [36]:
res = rfe.fit_transform(X_tr, y)
display(res.shape)
res

(800, 50)

array([[-0.98555744, -0.94218651, -0.91241606, ..., -0.41626469,
        -0.52135831, -0.72668962],
       [-0.48479877, -0.94218651, -0.51386057, ..., -0.10954899,
        -0.18003271, -0.70651232],
       [ 0.42451538, -0.94218651,  0.42528182, ...,  0.67405053,
         0.73602435, -0.6731154 ],
       ...,
       [-0.16049792, -0.94218651,  0.13418659, ...,  4.39362648,
         2.1076934 ,  4.87819432],
       [-0.16049792, -0.94218651,  0.31864298, ...,  5.8291638 ,
         2.51608978,  4.87819432],
       [ 1.36562373, -0.94218651,  1.51760948, ...,  1.1636675 ,
         0.19750623,  1.93926564]])

In [37]:
help(RFE)

Help on class RFE in module sklearn.feature_selection._rfe:

class RFE(sklearn.utils._metadata_requests._RoutingNotSupportedMixin, sklearn.feature_selection._base.SelectorMixin, sklearn.base.MetaEstimatorMixin, sklearn.base.BaseEstimator)
 |  RFE(estimator, *, n_features_to_select=None, step=1, verbose=0, importance_getter='auto')
 |  
 |  Feature ranking with recursive feature elimination.
 |  
 |  Given an external estimator that assigns weights to features (e.g., the
 |  coefficients of a linear model), the goal of recursive feature elimination
 |  (RFE) is to select features by recursively considering smaller and smaller
 |  sets of features. First, the estimator is trained on the initial set of
 |  features and the importance of each feature is obtained either through
 |  any specific attribute or callable.
 |  Then, the least important features are pruned from current set of features.
 |  That procedure is recursively repeated on the pruned set until the desired
 |  number of fea

In [38]:
pipe_rfe = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', RFE(LogisticRegression(max_iter = 1000),
                      n_features_to_select = 30,
                      step = 30
                     )), 
    ('model_', SVC(kernel='linear'))])

In [39]:
cv_res3 = cross_validate(pipe_rfe, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res3

{'fit_time': array([2.80941892, 3.46425509, 2.61953783, 3.7865479 , 2.94279408]),
 'score_time': array([0.00381494, 0.00375175, 0.0072782 , 0.00349212, 0.00352812]),
 'test_score': array([0.33333333, 0.84615385, 0.83870968, 0.48      , 0.75      ]),
 'train_score': array([0.91089109, 0.82105263, 0.91262136, 0.91836735, 0.96078431])}

In [40]:
cv_res3['train_score'].mean()

0.904743348115085

In [41]:
cv_res3['test_score'].mean()

0.6496393713813069

## С помощью L1 регуляризации

In [42]:
from sklearn.feature_selection import SelectFromModel

In [43]:
sel = SelectFromModel(LogisticRegression(penalty = 'l1', max_iter = 1000, solver = 'liblinear'), threshold = 1e-5)

In [44]:
# пример

res = sel.fit_transform(X_tr, y)
display(res.shape)
res

(800, 49)

array([[-1.39963712, -1.20562657, -0.94218651, ..., -0.52135831,
        -0.72668962, -0.81966779],
       [-1.39963712, -1.20562657, -0.94218651, ..., -0.18003271,
        -0.70651232, -0.81698899],
       [-1.39963712, -1.20562657, -0.94218651, ...,  0.73602435,
        -0.6731154 , -0.81341726],
       ...,
       [ 1.61195431, -1.20562657, -0.94218651, ...,  2.1076934 ,
         4.87819432,  4.18343684],
       [ 1.61195431, -1.20562657, -0.94218651, ...,  2.51608978,
         4.87819432,  4.18343684],
       [ 1.61195431, -1.20562657, -0.94218651, ...,  0.19750623,
         1.93926564,  2.6404483 ]])

In [45]:
pipe_lasso =  Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectFromModel(LogisticRegression(penalty = 'l1', max_iter = 1000, solver = 'liblinear'), 
                                  threshold=1e-5)), 
    ('model_', SVC(kernel = 'linear'))])

In [46]:
cv_res4 = cross_validate(pipe_lasso, X, y, cv = 5, scoring = 'f1', return_train_score = True)
cv_res4

{'fit_time': array([0.09396195, 0.07443213, 0.06615591, 0.06038713, 0.07207608]),
 'score_time': array([0.00362277, 0.00331807, 0.00351   , 0.00359988, 0.00381708]),
 'test_score': array([0.44444444, 0.83333333, 0.64285714, 0.58333333, 0.68421053]),
 'train_score': array([0.92307692, 0.875     , 0.94117647, 0.97029703, 0.92929293])}

In [47]:
cv_res4['train_score'].mean()

0.9277686705322117

In [48]:
cv_res4['test_score'].mean()

0.6376357560568087

# Нелинейные классификаторы

- SVM с ядром
- Наивный байесовский классификатор
- Метод k ближайших соседей

In [49]:
help(SVC)

Help on class SVC in module sklearn.svm._classes:

class SVC(sklearn.svm._base.BaseSVC)
 |  SVC(*, C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)
 |  
 |  C-Support Vector Classification.
 |  
 |  The implementation is based on libsvm. The fit time scales at least
 |  quadratically with the number of samples and may be impractical
 |  beyond tens of thousands of samples. For large datasets
 |  consider using :class:`~sklearn.svm.LinearSVC` or
 |  :class:`~sklearn.linear_model.SGDClassifier` instead, possibly after a
 |  :class:`~sklearn.kernel_approximation.Nystroem` transformer or
 |  other :ref:`kernel_approximation`.
 |  
 |  The multiclass support is handled according to a one-vs-one scheme.
 |  
 |  For details on the precise mathematical formulation of the provided
 |  kernel functions and how `g

In [50]:
pipe_lasso2 =  Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols = cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree = 4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectFromModel(LogisticRegression(penalty = 'l1', max_iter = 1000, solver = 'liblinear'), 
                                  threshold = 1e-5)), 
    ('model_', SVC(kernel = 'rbf'))])

In [51]:
cv_res5 = cross_validate(pipe_lasso2, X, y, cv = 5, scoring = 'f1', return_train_score=True)
cv_res5

{'fit_time': array([0.10290313, 0.07748914, 0.07475591, 0.06413484, 0.07471895]),
 'score_time': array([0.00723195, 0.00482297, 0.00518322, 0.00473404, 0.00477505]),
 'test_score': array([0.35294118, 0.7       , 0.5       , 0.42105263, 0.64864865]),
 'train_score': array([0.90909091, 0.81818182, 0.84782609, 0.90526316, 0.92929293])}

In [52]:
cv_res5['train_score'].mean()

0.881930980283383

In [53]:
cv_res5['test_score'].mean()

0.5245284913396369

Обучите наивный байесовский классификатор и метод k ближайших соседей вместо SVM в пайплайне выше. 

In [54]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB

In [ ]:
# your code here

С помощью GridSearch подберите гиперпараметр KNN (число соседей) внутри пайплайна.

In [ ]:
# your code here

# Сохранение и загрузка модели

In [55]:
import pickle

Обучим лучшую модель на всех данных и сохраним её в файл.

In [ ]:
#your code here

model = ...

Сохранение модели в файл

In [ ]:
filename = 'best_model.sav'
pickle.dump(model, open(filename, 'wb'))

Загрузка модели из файла

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))

Почитать подробнее про сохранение модели в файл и загрузку из файла тут: https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/